In [1]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os
import time as tm

In [2]:
#Reading the current main directory
main_directory = os.getcwd()

In [3]:
#Rreading the name of the subfoldeer
subfolder_with_path = [f.path for f in os.scandir(main_directory) if f.is_dir()]
subfolder_name = [f.name for f in os.scandir(main_directory) if f.is_dir()]

In [4]:
#Counting number of files on subfolder
list = os.listdir(subfolder_name[1]) 
number_files = len(list)

In [5]:
# Changing directory to the folder
os.chdir(subfolder_with_path[1])

In [ ]:
#Using a loop to process all files and populate the main data-frame

# Starting process timming
start = tm.time()

# Creating main data-frame
df_files_level = [] #This array will contain data frames from each file as elements

i = 0
while i < number_files:
    
    ##############################################
    # Processing each file
    ##############################################
    # Saving file's name
    file = list[i]
    
    # Reading number and names of sheets
    df_full = pd.read_excel(file, sheet_name = None)
    sheets_names = df_full.keys()
    numer_of_sheets = len(sheets_names)
    
    df_sheets_level = [" "]*number_of_sheets #This array will contain data frames from each sheets as elements
    
    # Reading files
    ThreeCT_data = pd.ExcelFile(file)

    # Saving group's name
    group_name = file[0:8]

    for j in range(0, numer_of_sheets):
        
        ##############################################
        # Processing each sheet
        ##############################################
        
        # Reading sheets within files
        df_sheets_level[j] = ThreeCT_data.parse(j,header = None)

        #Dropping the frist 13 rows and the row between the last reading and the "sum"
        rows_to_drop = np.arange(13)
        df_sheets_level[j] = df_sheets_level[j].drop(df_sheets_level[j].index[rows_to_drop])


        #locating empty and iso columns
        e = df_sheets_level[j].loc[13].where(df_sheets_level[j].loc[13].str.contains('empty')).dropna(how='all')
        e.index #To getthe column value
        empty = e.index[0]
        s = df_sheets_level[j].loc[13].where(df_sheets_level[j].loc[13].str.contains('iso')).dropna(how='all')
        s.index #To getthe column value
        iso = s.index[0]

        e2 = df2.loc[13].where(df2.loc[13].str.contains('empty')).dropna(how='all')
        e2.index #To getthe column value
        empty2 = e2.index[0]
        s2 = df2.loc[13].where(df2.loc[13].str.contains('iso')).dropna(how='all')
        s2.index #To getthe column value
        iso2 = s2.index[0]

        e3 = df3.loc[13].where(df3.loc[13].str.contains('empty')).dropna(how='all')
        e3.index #To getthe column value
        empty3 = e3.index[0]
        s3 = df3.loc[13].where(df3.loc[13].str.contains('iso')).dropna(how='all')
        s3.index #To getthe column value
        iso3 = s3.index[0]

        e4 = df4.loc[13].where(df4.loc[13].str.contains('empty')).dropna(how='all')
        e4.index #To getthe column value
        empty4 = e4.index[0]
        s4 = df4.loc[13].where(df4.loc[13].str.contains('iso')).dropna(how='all')
        s4.index #To getthe column value
        iso4 = s4.index[0]

        # Categorizing "duration" labels to iso or empty
        df1.iloc[1:2, empty1:empty1+1] = 'Duration_empty_(s)'
        df1.iloc[1:2, iso1:iso1+1] = 'Duration_iso_(s)'

        df2.iloc[1:2, empty2:empty2+1] = 'Duration_empty_(s)'
        df2.iloc[1:2, iso2:iso2+1] = 'Duration_iso_(s)'

        df3.iloc[1:2, empty3:empty3+1] = 'Duration_empty_(s)'
        df3.iloc[1:2, iso3:iso3+1] = 'Duration_iso_(s)'

        df4.iloc[1:2, empty4:empty4+1] = 'Duration_empty_(s)'
        df4.iloc[1:2, iso4:iso4+1] = 'Duration_iso_(s)'

        #Dropping 13th row, now the 0th row
        df1 = df1.drop(df1.index[0])
        df2 = df2.drop(df2.index[0])
        df3 = df3.drop(df3.index[0])
        df4 = df4.drop(df4.index[0])

        #Creating and assigning new header
        new_header = df1.iloc[0] #grab the first row for the header
        df1 = df1[1:] #take the data less the header row
        df1.columns = new_header #set the header row as the df header
        df2 = df2[1:] #take the data less the header row
        df2.columns = new_header #set the header row as the df header
        df3 = df3[1:] #take the data less the header row
        df3.columns = new_header #set the header row as the df header
        df4 = df4[1:] #take the data less the header row
        df4.columns = new_header #set the header row as the df header

        # Selecting columns of av verlocity and duration for iso and empty
        df1 = df1[["Avg. velocity (pixels/s)", "Duration_empty_(s)","Duration_iso_(s)"]]
        df1.columns=["AVG_VEL_pixels/s)", "DURATION_EMPTY_s", "DURATION_ISO_s"]
        df1.reset_index(drop=True, inplace=True)

        df2 = df2[["Avg. velocity (pixels/s)", "Duration_empty_(s)","Duration_iso_(s)"]]
        df2.columns=["AVG_VEL_pixels/s)", "DURATION_EMPTY_s", "DURATION_ISO_s"]
        df2.reset_index(drop=True, inplace=True)

        df3 = df3[["Avg. velocity (pixels/s)", "Duration_empty_(s)","Duration_iso_(s)"]]
        df3.columns=["AVG_VEL_pixels/s)", "DURATION_EMPTY_s", "DURATION_ISO_s"]
        df3.reset_index(drop=True, inplace=True)

        df4 = df4[["Avg. velocity (pixels/s)", "Duration_empty_(s)","Duration_iso_(s)"]]
        df4.columns=["AVG_VEL_pixels/s)", "DURATION_EMPTY_s", "DURATION_ISO_s"]
        df4.reset_index(drop=True, inplace=True)

        #Removing the Nan row between the last reading a the 'sum'
        df1 = df1.dropna(how='all')
        df2 = df2.dropna(how='all')
        df3 = df3.dropna(how='all')
        df4 = df4.dropna(how='all')


        # Creating time-column vector based on measurements' size
        unit = 20/(len(df1)-1)
        TIME_minutes = np.arange(unit,21,unit)
        TIME_minutes = np.append(TIME_minutes, "SUM")

          # Adding time in minutes' column
        df1.insert(0, 'TIME_STEP_minutes', TIME_minutes)
        df2.insert(0, 'TIME_STEP_minutes', TIME_minutes)
        df3.insert(0, 'TIME_STEP_minutes', TIME_minutes)
        df4.insert(0, 'TIME_STEP_minutes', TIME_minutes)

        #Adding mouse name
        df1.insert(0, 'MOUSE_NAME', [group_name + '_1']*len(df1))
        df2.insert(0, 'MOUSE_NAME', [group_name + '_2']*len(df1))
        df3.insert(0, 'MOUSE_NAME', [group_name + '_3']*len(df1))
        df4.insert(0, 'MOUSE_NAME', [group_name + '_4']*len(df1))

        #Summations a diffrences of duration (empty vs iso)
        sum1 = df1["DURATION_EMPTY_s"] + df1["DURATION_ISO_s"]
        sum2 = df2["DURATION_EMPTY_s"] + df2["DURATION_ISO_s"]
        sum3 = df3["DURATION_EMPTY_s"] + df3["DURATION_ISO_s"]
        sum4 = df4["DURATION_EMPTY_s"] + df4["DURATION_ISO_s"]

        diff1 = df1["DURATION_ISO_s"] - df1['DURATION_EMPTY_s']
        diff2 = df2["DURATION_ISO_s"] - df2['DURATION_EMPTY_s']
        diff3 = df3["DURATION_ISO_s"] - df3['DURATION_EMPTY_s']
        diff4 = df4["DURATION_ISO_s"] - df4['DURATION_EMPTY_s']

        # Changin any sum value from zero to 10^20
        sum1.loc[sum1 == 0] = pow(10,20)
        sum2.loc[sum2 == 0] = pow(10,20)
        sum3.loc[sum3 == 0] = pow(10,20)
        sum4.loc[sum4 == 0] = pow(10,20)

        # Calculating relative and normalized socialindexes
        df1['SOL_IDX_REL'] = 100 * diff1/sum1
        df2['SOL_IDX_REL'] = 100 * diff2/sum2
        df3['SOL_IDX_REL'] = 100 * diff3/sum3
        df4['SOL_IDX_REL'] = 100 * diff4/sum4

        df1['SOL_IDX_NORM'] = 100 * diff1/(5*60)
        df2['SOL_IDX_NORM'] = 100 * diff2/(5*60)
        df3['SOL_IDX_NORM'] = 100 * diff3/(5*60)
        df4['SOL_IDX_NORM'] = 100 * diff4/(5*60)

        # Changin any sum value from zero to 10^20


#         data_frames.append(df1)
#         data_frames.append(df2)
#         data_frames.append(df3)
#         data_frames.append(df4)

    ##########################################################################################################################
    

    i += 1
    

appended_data = pd.concat(data_frames)
end = tm.time()

# Priting running time and files processed
print(i,' Files processed ')
print(' ')
print(' Execution time:', round((end - start),2), 'seconds') 

# Exporting dt to csv
appended_data.to_excel('3CT_data.xlsx',index=False)


In [ ]:
# Going back to the main directory
os.chdir(main_directory)

In [10]:
# Saving file's name
file = list[0] #i = 0
# Creating main data-frame
df_files_level = [] #This array will contain data frames as elements


# Reading number and names of sheets
df_full = pd.read_excel(file, sheet_name = None)
sheets_names = df_full.keys()
numer_of_sheets = len(sheets_names)
df_sheets_level = [" "] * numer_of_sheets

# Reading files
ThreeCT_data = pd.ExcelFile(file)

# Saving group's name
group_name = file[0:8]


df_sheets_level[0] = ThreeCT_data.parse(0,header = None) # Reading sheets within files j = 0
rows_to_drop = np.arange(13)
df_sheets_level[0] = df_sheets_level[0].drop(df_sheets_level[0].index[rows_to_drop])

#locating empty and iso columns
e = df_sheets_level[0].loc[13].where(df_sheets_level[0].loc[13].str.contains('empty')).dropna(how='all')
e.index #To getthe column value
empty = e.index[0]
s = df_sheets_level[0].loc[13].where(df_sheets_level[0].loc[13].str.contains('iso')).dropna(how='all')
s.index #To getthe column value
iso = s.index[0]


numpy.int64